<a href="https://colab.research.google.com/github/dannynew111/erp-clustering-smart-meter-data/blob/main/survey_data_preparation_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the notebook I used to prepare the final clean dataset with consumption, intrinsic, economic and survey features ('LCL_Report_Ready_Dataset.csv') for final strategic insights (in the final notebook)

In [ ]:
# Reproducibility design:
#  Read-only merge: personas (engineered features + persona labels) ⟶ LEFT join ⟵ raw survey
# Strict ID normalisation to "D####" + de-duplication on both inputs
# Deterministic recoding of survey text → numeric features via explicit maps
# Deterministic persona mapping (numeric code and name) with safety guards
# Human-readable survey column names for downstream analysis/plots
# Final export columns: [ID, persona (numeric), engineered, survey, persona_name]

#Import required libraries
import re, os
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

pd.set_option('display.max_columns', None)

#Data paths
data_loaded_dir = '/content/drive/MyDrive/ERP/data_loaded/'
data_raw_dir    = '/content/drive/MyDrive/ERP/data_raw/'

# Inputs (from clustering + raw survey)
personas_path = os.path.join(data_loaded_dir, 'final_persona_dataset_v2_cleaned.csv')
survey_path = os.path.join(data_raw_dir,    'survey_answers.csv')

# Output - the defeinitive final dataset with all relevant features for analysis
out_lcl_path = os.path.join(data_loaded_dir, 'LCL_Report_Ready_Dataset.csv')

Mounted at /content/drive


In [79]:
# Load, normalise IDs, de-duplicate, and join
#Key as there were 14 duplicates from a past error, fixed here
def normalise_id(x: str) -> str:
    #Force IDs into D#### format (e.g., 31 -> D0031, D31 -> D0031, d0042 -> D0042).
    #prevents mismatches from whitespace/case/format inconsistencies
    if pd.isna(x): return np.nan
    s = str(x).strip().upper()
    m = re.search(r'(\d+)', s)
    if not m: return np.nan
    return f"D{int(m.group(1)):04d}"

# Load data as text (max control - helps me make required changes)
print("Loading")
df_p = pd.read_csv(personas_path, dtype=str) ## engineered features + persona labels here
df_s = pd.read_csv(survey_path,   dtype=str) #raw survey data
print(f"  personas: {df_p.shape}, survey: {df_s.shape}")

# Normalise IDs on both frames (critical to avoid merge fragmentation)
for df in (df_p, df_s):
    df['Household_id'] = df['Household_id'].apply(normalise_id)

# Drop duplicated HouseHold rows within each source (keep the 1st occurrence)
# enforces a single row per house prior to merge; prevents 1→N joins.
dup_p = df_p.duplicated('Household_id').sum()
dup_s = df_s.duplicated('Household_id').sum()
if dup_p:
    print(f"{dup_p} duplicate persona IDs; keeping first")
    df_p = df_p.drop_duplicates('Household_id', keep='first')
if dup_s:
    print(f"{dup_s} duplicate survey IDs; keeping first")
    df_s = df_s.drop_duplicates('Household_id', keep='first')

# LEFT JOIN: keep every household from personas (1025 expected), even if no survey
# validate='one_to_one' here is safe after explicit de-duplication
expected_personas = df_p['Household_id'].nunique()
df_m = pd.merge(df_p, df_s, on='Household_id', how='left', validate='one_to_one')
print(f"Merge complete. Persona rows: {len(df_m)} (unique HH: {df_m['Household_id'].nunique()}; expected {expected_personas})")


Loading
  personas: (1025, 24), survey: (2785, 345)
404 duplicate survey IDs; keeping first
Merge complete. Persona rows: 1025 (unique HH: 1025; expected 1025)


In [ ]:
# Encode survey answers to numeric features; finish persona columns

# Evidence-based encodings
# Likert scales: ordered (Agree/Disagree; Often/Never)
# Yes/No: binary
# Climate intent: domain-informed ordinal mapping
likert_5_agree = {'Strongly agree': 5, 'Agree': 4, 'Neither agree nor disagree': 3, 'Disagree': 2, 'Strongly disagree': 1} #as explained in the main report it is 1-5 interval scale
likert_5_often = {'Always': 5, 'Usually': 4, 'Occasionally': 3, 'Rarely': 2, 'Never': 1, 'Half the time': 2.5}
yes_no         = {'Yes': 1, 'No': 0}
climate_map    = {"I'd like to do a lot more to help the environment": 2,
                  "I'd like to do a bit more to help the environment": 1,
                  "I'm happy with what I do at the moment": 0}
work_freq_map  = {'Most/all weekdays': 4, 'About half the time': 3, 'Occasionally': 2, 'Never': 1}

# Central processing plan: (raw survey column) to (internal feature name, encoding map)
# I made this change, creating a single source of truth for all survey recodes,
#so the outputted file would have easay to recognise names and anyone reproducing the
#study wouldn't have to wonder what 'dem_insul_glaze' was - ie it improves the readbility
processing_plan = {
    'Q234': ('dem_work_from_home', work_freq_map),
    'Q335': ('dem_climate_concern', climate_map),
    'Q240': ('dem_insul_glaze', yes_no),
    'Q241': ('dem_insul_roof',  yes_no),
    'Q242': ('dem_insul_wall',  yes_no),

    'Q01': ('shift_wmachine',  likert_5_often),
    'Q02': ('shift_tdryer',    likert_5_often),
    'Q03': ('shift_dwasher',   likert_5_often),

    'Q39': ('routine_wmachine', yes_no),
    'Q40': ('routine_tdryer',   yes_no),
    'Q41': ('routine_dwasher',  yes_no),

    'Q137': ('con_not_home',         likert_5_agree),
    'Q138': ('con_work_inflexible',  likert_5_agree),
    'Q139': ('con_children_routine', likert_5_agree),
    'Q140': ('con_fixed_routine',    likert_5_agree),
    'Q142': ('con_comfort_loss',     likert_5_agree),
    'Q146': ('con_savings_too_small', likert_5_agree),
    'Q147': ('con_notice_period',     likert_5_agree),

    'Q180': ('att_control_bill',      likert_5_agree),
    'Q186': ('att_convenience_focus', likert_5_agree),
    'Q187': ('att_hassle',            likert_5_agree),
    'Q189': ('att_planning_talk',     likert_5_agree),
    'Q197': ('att_want_to_stay',      likert_5_agree),
}

In [ ]:
# Apply the plan to create internal numeric features
# If a question is absent, I create a warning but do not fail the build
df_final = df_m.copy()
for raw_col, (clean_col, enc_map) in processing_plan.items():
    if raw_col in df_final.columns:
        df_final[clean_col] = df_final[raw_col].str.strip().map(enc_map)
    else:
        print(f"• Skipping {raw_col} (not in survey)")

# Accommodation: combines mutually exclusive house/flat answers into one ordinal feature.
# Rule: if a house type exists, use it; otherwise use flat type.
#This is explained further in the main report (specifically used as an example)
#as it takes multiple complex survey answers and forms one, easy to interpret scale of how in control of energy the user is
house_map = {'Detached': 5, 'Semi-detached': 4, 'Terraced - end': 3, 'Terraced - middle': 2}
flat_map  = {'In a purpose built block or tenement': 1, 'Part of a converted house/building': 0}
df_final['temp_house'] = df_final.get('Q235', pd.Series(index=df_final.index)).astype(str).str.strip().map(house_map)
df_final['temp_flat']  = df_final.get('Q236', pd.Series(index=df_final.index)).astype(str).str.strip().map(flat_map)
df_final['dem_accom_type'] = df_final['temp_house'].fillna(df_final['temp_flat'])

# Household size from Q213 (extract number from text; coerce to numeric)
df_final['dem_hh_size'] = pd.to_numeric(df_final.get('Q213', pd.Series(index=df_final.index)).astype(str).str.extract(r'(\d+)', expand=False),
                                        errors='coerce')

# Keep persona_name if present; otherwise, try persona → persona_name
persona_name_col = None
if 'persona_name' in df_final.columns:
    persona_name_col = 'persona_name'

# Ensure a numeric 'persona' column exists and is integer-coded
# Map names -> cluster numbers I use in the report
# Ensure BOTH persona (numeric code) and persona_name (label) exist
# If there is already a persona_name, great; otherwise I build it from persona
code_to_name = {
    0: 'Structurally Constrained',
    1: 'Low-Volatility Consumers',
    2: 'Engaged Shifters',
    3: 'Contrarians'
}
name_to_code = {v: k for k, v in code_to_name.items()}

# If only names exist, create numeric codes
if 'persona' not in df_final.columns and 'persona_name' in df_final.columns:
    df_final['persona'] = df_final['persona_name'].map(name_to_code)

# If only codes exist, create names
if 'persona_name' not in df_final.columns and 'persona' in df_final.columns:
    # persona might be str; coerce to numeric first for safety
    df_final['persona'] = pd.to_numeric(df_final['persona'], errors='coerce')
    df_final['persona_name'] = df_final['persona'].map(code_to_name)

# Final type safety: persona as nullable integer (preserves NaNs if any)
df_final['persona'] = pd.to_numeric(df_final['persona'], errors='coerce').astype('Int64')

In [80]:
# Rename survey columns to friendly names and assemble final export

# Engineered features = everything in personas file except ID and any persona label columns.
# from df_p as ensures I export the original engineered columns produced by clustering,
# not any accidental columns created by the merge.
engineered_cols = [c for c in df_p.columns if c not in ['Household_id', 'persona', 'persona_name']]

#Map confusing names to the final, report-friendly ones used in my visualisations and report
lcl_rename_map = {
    'dem_work_from_home': 'Frequency of Working From Home',
    'dem_climate_concern': 'Stated Climate Concern',
    'dem_insul_glaze': 'Insulation: Double Glazing',
    'dem_insul_roof':  'Insulation: Roof/Loft',
    'dem_insul_wall':  'Insulation: Wall',
    'shift_wmachine':   'Self-Reported Shift of Washing Machine',
    'shift_tdryer':     'Self-Reported Shift of Tumble Dryer',
    'shift_dwasher':    'Self-Reported Shift of Dishwasher',
    'routine_wmachine': 'Has Fixed Routine for Washing Machine',
    'routine_tdryer':   'Has Fixed Routine for Tumble Dryer',
    'routine_dwasher':  'Has Fixed Routine for Dishwasher',
    'con_not_home':         'Limited by Being Away From Home',
    'con_work_inflexible':  'Limited by Inflexible Work Schedule',
    'con_children_routine': "Limited by Children's Routines",
    'con_fixed_routine':    'Limited by Fixed Household Habits',
    'con_comfort_loss':     'Unwillingness to Reduce Comfort',
    'con_savings_too_small':'Perceived Low Financial Reward',
    'con_notice_period':    'Insufficient Notice Period for Alerts',
    'att_control_bill':      'Felt More in Control of Bill',
    'att_convenience_focus': 'Prioritised Convenience',
    'att_hassle':            'Perceived Inconvenience of Shifting',
    'att_planning_talk':     'Planned Energy Use Around Tariffs',
    'att_want_to_stay':      'Desire to Remain on Tariff',
    'dem_accom_type':        'Accommodation Type',
    'dem_hh_size':           'Household Size'
}

# Only include survey columns that were *actually created* this run (guards missing questions)
internal_created = [k for k in lcl_rename_map.keys() if k in df_final.columns]
friendly_created = [lcl_rename_map[k] for k in internal_created]

# Work on a copy and rename survey columns to their friendly labels
df_export = df_final.copy()
df_export.rename(columns=lcl_rename_map, inplace=True)

# Final column order:
#   1) Household_id
#   2) persona (numeric; the modelling target)
#   3) engineered features (from personas file)
#   4) survey features created this run (friendly names)
#   5) persona_name (for plotting/legends) at the very end
base_cols = ['Household_id', 'persona']
export_cols = base_cols + engineered_cols + friendly_created

if 'persona_name' in df_export.columns:
    export_cols += ['persona_name']

# only keep columns that actually exist (robust if a survey column is absent)
export_cols = [c for c in export_cols if c in df_export.columns]
df_export = df_export.loc[:, export_cols].copy()

# Type safety: persona must be numeric for Notebook 6 (modelling).
df_export['persona'] = pd.to_numeric(df_export['persona'], errors='coerce').astype('Int64')

# cast survey columns to numeric where possible
for c in friendly_created:
    if c in df_export.columns:
        df_export[c] = pd.to_numeric(df_export[c], errors='coerce')

In [81]:
#Final checks and save
# Sort and assert 1 row per Household (guard against accidental multiplies)
df_export = df_export.sort_values('Household_id').reset_index(drop=True)
assert df_export['Household_id'].is_unique, "Duplicate Household_id rows remain!"

out_lcl_path = '/content/drive/MyDrive/ERP/data_loaded/LCL_Report_Ready_Dataset.csv'
df_export.to_csv(out_lcl_path, index=False)
print(f"\nSaved LCL output (ID + persona + engineered + survey + persona_name-last) to: {out_lcl_path}")

print("\nPreview:")
display(df_export.head(10))


Saved LCL output (ID + persona + engineered + survey + persona_name-last) to: /content/drive/MyDrive/ERP/data_loaded/LCL_Report_Ready_Dataset.csv

Preview:


,Household_id,persona,base_weekday_offpeak,base_weekday_peak,base_weekend_offpeak,base_weekend_peak,trial_weekday_high,trial_weekday_low,trial_weekday_normal,trial_weekend_high,trial_weekend_low,trial_weekend_normal,intrinsic_daily_volatility,intrinsic_load_factor,base_peak_to_offpeak_ratio,base_day_to_night_ratio,base_morning_vs_evening_peak_ratio,peak_reduction_weekday,peak_reduction_weekend,peak_reduction_pct_weekday,peak_reduction_pct_weekend,load_shift_weekday,estimated_cost_savings_pct,economic_rationality_score,Frequency of Working From Home,Stated Climate Concern,Insulation: Double Glazing,Insulation: Roof/Loft,Insulation: Wall,Self-Reported Shift of Washing Machine,Self-Reported Shift of Tumble Dryer,Self-Reported Shift of Dishwasher,Has Fixed Routine for Washing Machine,Has Fixed Routine for Tumble Dryer,Has Fixed Routine for Dishwasher,Limited by Being Away From Home,Limited by Inflexible Work Schedule,Limited by Children's Routines,Limited by Fixed Household Habits,Unwillingness to Reduce Comfort,Perceived Low Financial Reward,Insufficient Notice Period for Alerts,Felt More in Control of Bill,Prioritised Convenience,Perceived Inconvenience of Shifting,Planned Energy Use Around Tariffs,Desire to Remain on Tariff,Accommodation Type,Household Size,persona_name
0,D0000,1,0.1119645924627519,0.1702737782448765,0.1248924162257495,0.1598399470899471,0.156927659574468,0.1395532516493873,0.1277990178246635,0.1621999999999999,0.183983286908078,0.1401180380472152,1.7195783926575012,0.5285085904882318,1.520768787085391,1.2011196263183468,1.0652700784427822,0.0100941111813827,-0.0058364951398391,5.928130569558685,-3.651439287678111,0.0256422162122715,2.82018809593104,1.821488712705328,1.0,1.0,1.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,1.0,5.0,4.0,1.0,2.0,1.0,5.0,1.0,1.0,4.0,5.0,1.0,2.0,Low-Volatility Consumers
1,D0001,1,0.0948696855843385,0.12021327014218,0.0930570436507936,0.1119717261904762,0.1109106382978723,0.0994882186616399,0.1069711660905948,0.1177384615384615,0.1026768802228412,0.1080330559408229,1.4084791346903875,0.496032835589617,1.2671276633213784,1.0275250427128135,1.0735942066614146,0.006050624355282,-0.0092431775777715,5.033199759168257,-8.254847311699837,0.0026720901029376,4.254401444955662,1.721271431940157,1.0,0.0,1.0,NaN,NaN,4.0,NaN,NaN,0.0,NaN,NaN,4.0,NaN,1.0,1.0,1.0,1.0,1.0,4.0,2.0,1.0,4.0,4.0,1.0,1.0,Low-Volatility Consumers
2,D0002,2,0.300736797274276,0.6002044075375279,0.4507732804232804,0.6996301587301588,0.4928042553191489,0.368057088487155,0.3803742950700382,0.5310777777777778,0.5228495821727019,0.4139349071739628,8.111154615179897,0.4206190842890846,1.99577310526796,1.7625358931246684,0.8578869023313771,0.1041481447293533,0.1650759387225948,17.352083706916865,23.594709392619126,0.0653738482385152,0.8054748310225106,1.5320458934797672,4.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,Engaged Shifters
3,D0003,0,0.1970640643623361,0.15843125,0.2314926900584795,0.1514561403508771,0.0994872340425531,0.2321856738925541,0.1804118395926161,0.1041666666666666,0.2727493036211699,0.2162711202938475,1.4632729666438689,0.4493153383443455,0.8039540164699028,0.692283639618822,1.0460467537744744,0.0556920084684211,0.0438130314544242,35.15193937372042,28.927676405961236,0.0331751665558542,14.243652678764471,4.597323799512262,1.0,1.0,1.0,0.0,0.0,NaN,5.0,NaN,NaN,1.0,NaN,2.0,NaN,4.0,1.0,4.0,1.0,2.0,5.0,2.0,4.0,4.0,4.0,1.0,1.0,Structurally Constrained
4,D0004,0,0.0968837304847986,0.2600184501845018,0.0998511904761904,0.1914374999999999,0.1429425531914893,0.1034905660377358,0.1104392174704276,0.1304481481481481,0.1063175487465181,0.0998260371304148,1.4505039855757214,0.6192372917811914,2.683791851238096,1.2021501504380483,1.3582348910198414,0.1138238895039868,0.0575129096220655,43.77514429140624,30.042499090865004,0.0046603925785733,0.5909770139880233,1.4439670184371018,2.0,1.0,1.0,1.0,1.0,4.0,NaN,NaN,1.0,NaN,NaN,4.0,2.0,4.0,4.0,1.0,2.0,2.0,5.0,5.0,2.0,1.0,5.0,2.0,4.0,Structur

In [ ]:
#Sanity Check 2

df = pd.read_csv('/content/drive/MyDrive/ERP/data_loaded/LCL_Report_Ready_Dataset.csv')

# 1) persona is numeric and unique mapping to names
print(df['persona'].dtype)
print(pd.crosstab(df['persona'], df.get('persona_name')))

# 2) confirm row count and engineered columns are there
print(len(df), "rows")
engineered_expected = ['base_weekday_offpeak','base_weekday_peak','base_weekend_offpeak',  # etc…
                       ]  # put your engineered list here if you want
print([c for c in engineered_expected if c not in df.columns], "missing engineered cols")

# 3) check a few survey columns exist and are numeric
survey_check = ['Planned Energy Use Around Tariffs', 'Frequency of Working From Home']
print(df[survey_check].dtypes)
print(df[survey_check].notna().sum())

int64
persona_name  Contrarians  Engaged Shifters  Low-Volatility Consumers  \
persona                                                                 
0                       0                 0                         0   
1                       0                 0                       351   
2                       0               121                         0   
3                      34                 0                         0   

persona_name  Structurally Constrained  
persona                                 
0                                  519  
1                                    0  
2                                    0  
3                                    0  
1025 rows
[] missing engineered cols
Planned Energy Use Around Tariffs    float64
Frequency of Working From Home       float64
dtype: object
Planned Energy Use Around Tariffs    626
Frequency of Working From Home       886
dtype: int64
